In [1]:
import numpy as np
import xarray as xr
import netCDF4
import os
import datetime
import matplotlib.pyplot as plt 
from matplotlib import cm
import shutil
from datetime import datetime, timedelta
import glob
import datetime as dt
from os import path
import fsspec
import json
import sys
import yaml

from functions_verification import *

## Time Range Definition

In [2]:
today= datetime.today()

In [3]:
today_string=datetime.today().strftime('%Y%m%d')
today_string2=datetime.today().strftime('%Y-%m-%d')

In [4]:
startyear=2018
endyear=today.year

## Data Read In

In [5]:
#with open(r"scripts/input_python.yaml", "r") as f:
#    directories = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
with open(r"/Volumes/lexplore_hd/scripts/input_python.yaml", "r") as f:
    directories = yaml.load(f, Loader=yaml.FullLoader)

In [7]:
os.path.abspath("scripts/input_python.yaml")

'/Users/mwegmann/scripts/input_python.yaml'

In [8]:
log("Creating directories")
for d in directories.values():
    if not os.path.exists(d):
        os.makedirs(d)

10:37:30.886488   Creating directories


In [9]:
# defining folders
output_folder=directories["plot_output_folder"]
tchain_folder_netcdf= directories["tchain_folder"]
ctd_folder_netcdf = directories["ctd_folder"]
pp_mooring_par_folder_netcdf=directories["pp_mooring_folder_par"]
pp_mooring_do_folder_netcdf=directories["pp_mooring_folder_do"]
thetis_folder_netdf = directories["thetis_folder"]
idronaut_folder_netcdf = directories["idronaut_folder"]
scripts_folder=directories["scripts_folder"]

In [10]:
# get faulty dates for idronaut
f = open(os.path.join(scripts_folder, "idronaut_faulty.json"))
dates2exclude = json.load(f)

In [11]:
log("read in netcdf data")
try:
    thetis_all=read_in_multiple_files(startyear=startyear,endyear=endyear,folder=thetis_folder_netdf,pr=False)
    pp_do_all=read_in_multiple_files(startyear=startyear,endyear=endyear,folder=pp_mooring_do_folder_netcdf,pr=False)
    pp_par_all=read_in_multiple_files(startyear=startyear,endyear=endyear,folder=pp_mooring_par_folder_netcdf,pr=False)
    tchain_all=read_in_multiple_files(startyear=startyear,endyear=endyear,folder=tchain_folder_netcdf,pr=False)
    tchain_all=tchain_all.temp.where(tchain_all.interp_temp<1)
    tchain_all.attrs["source"]="Temperature Chain"
    idronaut_all=read_in_multiple_files(startyear=startyear,endyear=endyear,folder=idronaut_folder_netcdf,pr=False)
    ctd_all=read_in_multiple_files(startyear=startyear,endyear=endyear,folder=ctd_folder_netcdf,pr=False)
except:
    log("Failed to read in netcdf data")

10:37:30.921585   read in netcdf data


## Manual deletion of questionable data

In [12]:
log("manual deletion of faulty outliers")
try:
    pp_do_all=clean_pp(pp_do_all,var="do")
    pp_par_all=clean_pp(pp_par_all,var="par")
    idronaut_all=clean_idronaut(idronaut_all,dates2exclude=dates2exclude)
except:
    log("Failed manual deletion of faulty outliers")

10:40:58.878921   manual deletion of faulty outliers


## PostProcessing

In [13]:
# define common depth
thetis_depths_limit=tchain_all.depth.values[3:35]*-1

In [14]:
manual_depths=[ -1.  ,  -1.5 ,  -2.  ,  -2.5 ,  -3.  ,  -4.  ,  -5.  ,
        -6.  ,  -7.  ,  -8.  ,  -9.  , -10.  ,
       -15.  , -21.  , -24.  ,
       -30.  , -36.  , -39.  , -45.  , -51.  ]

In [15]:
# change depth coordinates for easier plotting
log("change depth coordinates for easier plotting")
thetis_all.coords['depth'] = (thetis_all.coords['depth'] * -1)
idronaut_all.coords['Press'] = (idronaut_all.coords['Press'] * -1)
tchain_all.coords['depth'] = (tchain_all.coords['depth'] * -1)
ctd_all.coords['depth'] = (ctd_all.coords['depth'] * -1)
idronaut_all=idronaut_all.rename({'Press': 'depth'})

10:41:01.715772   change depth coordinates for easier plotting


In [16]:
#rename variables
log("rename variables")
idronaut_all=idronaut_all.rename({'Chl': 'chla','Sal': 'sal','Temp': 'temp','Cond': 'cond'})
ctd_all=ctd_all.rename({'Chl_A': 'chla','SALIN': 'sal','Temp': 'temp','Cond': 'cond','DO_mg': 'do'})

10:41:01.724765   rename variables


In [17]:
# create plotting dictionaries
idronaut_thetis_dict=["chla","temp","sal","cond"]
idronaut_ctd_dict=["chla","temp","pH"]
thetis_ctd_dict=["chla","temp","sal","cond","do"]
thetis_dict=["chla","temp","sal","cond","do","par"]
idronaut_dict=["chla","temp","sal","cond","pH"]
ctd_dict=["chla","temp","sal","cond","pH","do"]
cc_dict=["depth","date"]

In [18]:
# reducing depth information
# reduce to same depths
#log("reducing depth information")
#thetis_all_less_depths=thetis_all.sel(depth=thetis_depths_limit, method='nearest')
#ctd_all_less_depths=ctd_all.sel(depth=thetis_depths_limit, method='nearest')
#tchain_all_less_depths=tchain_all.sel(depth=thetis_depths_limit, method='nearest')
#idronaut_all_less_depths=idronaut_all.sel(depth=thetis_depths_limit, method='nearest')

In [19]:
# reducing depth information
# reduce to same depths
log("reducing depth information")
thetis_all_less_depths=thetis_all[thetis_dict].sel(depth=manual_depths, method='nearest')
ctd_all_less_depths=ctd_all[ctd_dict].sel(depth=manual_depths, method='nearest')
tchain_all_less_depths=tchain_all.sel(depth=manual_depths, method='nearest')
idronaut_all_less_depths=idronaut_all[idronaut_dict].sel(depth=manual_depths, method='nearest')

10:41:01.740605   reducing depth information


In [20]:
# night only values
log("create night only files")
thetis_all_less_depths_night=get_night(thetis_all_less_depths)
idronaut_all_less_depths_night=get_night(idronaut_all_less_depths)
ctd_all_less_depths_night=get_night(ctd_all_less_depths)

10:41:01.760173   create night only files


In [21]:
# reduce the temporal high resolution instruments to lower resolution
log("reduce the temporal high resolution instruments to lower resolution")
try:
    pp_par_all_30mins=pp_par_all.resample(time='30min').mean()
    tchain_all_30mins=tchain_all_less_depths.resample(time='30min').mean()  
    pp_do_all_30mins=pp_do_all.resample(time='30min').mean()
    pp_do_all_30mins.attrs["source"]="PP Mooring"
    pp_do_all_30mins.attrs["units"]=thetis_all.do.units
    pp_par_all_30mins.attrs["source"]="PP Mooring"
    pp_par_all_30mins.attrs["units"]=thetis_all.par.units
    tchain_all_30mins.attrs["source"]="Temperature Chain"
    tchain_all_30mins.attrs["units"]=thetis_all.temp.units
except:
    log("Failed to reduce the temporal high resolution instruments to lower resolution")

10:41:33.204599   reduce the temporal high resolution instruments to lower resolution


In [22]:
# 12 hour means
log("plot climatology and availability")
try:
    plot_clim(thetis_all,"temp",output_folder=output_folder,cmap="inferno",res="daily")
    plot_clim(thetis_all,"temp",output_folder=output_folder,cmap="inferno",res="monthly")
    plot_samples(thetis_all,"temp",output_folder=output_folder,cmap="viridis",res="monthly")
    plot_samples(idronaut_all,"temp",output_folder=output_folder,cmap="viridis",res="monthly")
    plot_samples(tchain_all,"tchain",output_folder=output_folder,cmap="viridis",res="monthly")
    plot_samples(ctd_all,"temp",output_folder=output_folder,cmap="viridis",res="monthly")
except:
    log("Failed to plot climatology and availability")

10:53:15.841084   plot climatology and availability


/Users/mwegmann/miniconda2/envs/temprec/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


Figure saved: Thetis CTD daily temperature Climatology.pdf
Figure saved: Thetis CTD monthly temperature Climatology.pdf
Figure saved: Thetis CTD monthly temperature sample size.pdf
Figure saved: Idronaut monthly temperature sample size.pdf
Figure saved: Temperature Chain monthly temperature sample size.pdf
Figure saved: LéXPLORE CTD monthly temperature sample size.pdf


In [23]:
sys.getsizeof(pp_do_all_30mins)

128

# Selecting matching observations

In [24]:
# 12 hour means
log("Selecting matching observations between instruments")
try:
    thetis_tchain_intersect_nearest,tchain_thetis_intersect_nearest=verification(thetis_all_less_depths,tchain_all_30mins)
    idronaut_tchain_intersect_nearest,tchain_idronaut_intersect_nearest=verification(idronaut_all_less_depths,tchain_all_30mins)
    thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest=verification(thetis_all_less_depths,ctd_all_less_depths)
    thetis_idronaut_intersect_nearest,idronaut_thetis_intersect_nearest=verification(thetis_all_less_depths,idronaut_all_less_depths)
    ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest=verification(ctd_all_less_depths,idronaut_all_less_depths)
    idronaut_tchain_intersect_nearest,tchain_idronaut_intersect_nearest=verification(idronaut_all_less_depths,tchain_all_30mins)
    thetis_idronaut_intersect_nearest_night,idronaut_thetis_intersect_nearest_night=verification(thetis_all_less_depths_night,idronaut_all_less_depths_night)
    thetis_pp_par_intersect_nearest,pp_par_thetis_intersect_nearest=verification(thetis_all_less_depths,pp_par_all_30mins)
    ctd_pp_do_intersect_nearest,pp_do_ctd_intersect_nearest=verification(ctd_all_less_depths,pp_do_all_30mins) 
    thetis_idronaut_intersect_nearest_night,idronaut_thetis_intersect_nearest_night=verification(thetis_all_less_depths_night,idronaut_all_less_depths_night)
    ctd_idronaut_intersect_nearest_night,idronaut_ctd_intersect_nearest_night=verification(ctd_all_less_depths_night,idronaut_all_less_depths_night)
    thetis_pp_do_intersect_nearest,pp_do_thetis_intersect_nearest=verification(thetis_all_less_depths,pp_do_all_30mins)
except:
    log("failed to select matching observations between instruments")

10:53:54.577581   Selecting matching observations between instruments


In [25]:
thetis_all = None
idronaut_all = None
ctd_all = None
pp_do_all= None
pp_par_all= None
tchain_all= None
pp_do_all_30mins= None
pp_par_all_30mins= None
tchain_all_30mins= None

In [26]:
# this vastly increases the plotting speed below
#log("load highres data into memory")
pp_do_thetis_intersect_nearest=pp_do_thetis_intersect_nearest.load()
pp_par_thetis_intersect_nearest=pp_par_thetis_intersect_nearest.load()
tchain_thetis_intersect_nearest=tchain_thetis_intersect_nearest.load()
tchain_idronaut_intersect_nearest=tchain_idronaut_intersect_nearest.load()
pp_do_ctd_intersect_nearest=pp_do_ctd_intersect_nearest.load()

/Users/mwegmann/miniconda2/envs/temprec/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/Users/mwegmann/miniconda2/envs/temprec/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/Users/mwegmann/miniconda2/envs/temprec/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/Users/mwegmann/miniconda2/envs/temprec/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/Users/mwegmann/miniconda2/envs/temprec/lib/python3.7/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


## Plotting

### Thetis vs Manual CTD

In [27]:
log("plotting verification Thetis vs Manual CTD")
try:
    for i in thetis_ctd_dict:
        for b in cc_dict:
            ver_plotting(thetis_ctd_intersect_nearest,ctd_thetis_intersect_nearest,output_folder=output_folder,var1=i,var2=i,cc=b)
except:
    log("Failed plotting verification Thetis vs Manual CTD")

10:56:04.368615   plotting verification Thetis vs Manual CTD
Figure saved: Thetis CTD vs LéXPLORE CTD Chlorophyll a depth.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD Chlorophyll a date.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD temperature depth.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD temperature date.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD salinity depth.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD salinity date.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD conductivity depth.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD conductivity date.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD dissolvedoxygen depth.pdf
Figure saved: Thetis CTD vs LéXPLORE CTD dissolvedoxygen date.pdf


### CTD vs Idronaut

In [28]:
log("plotting verification manual CTD vs Idronaut")
try:
    for i in idronaut_ctd_dict:
        for b in cc_dict:
            ver_plotting(ctd_idronaut_intersect_nearest,idronaut_ctd_intersect_nearest,output_folder=output_folder,var1=i,var2=i,cc=b)
except:
    log("Failed plotting verification manual CTD vs Idronaut")

10:57:25.369859   plotting verification manual CTD vs Idronaut
Figure saved: LéXPLORE CTD vs Idronaut chlorophyll A depth.pdf
Figure saved: LéXPLORE CTD vs Idronaut chlorophyll A date.pdf
Figure saved: LéXPLORE CTD vs Idronaut temperature depth.pdf
Figure saved: LéXPLORE CTD vs Idronaut temperature date.pdf
Figure saved: LéXPLORE CTD vs Idronaut pH depth.pdf
Figure saved: LéXPLORE CTD vs Idronaut pH date.pdf


### Idronaut vs Temperature chain

In [29]:
log("plotting verification Idronaut vs Temperature Chain")
try:
    for b in cc_dict:
        ver_plotting_tchain(idronaut_tchain_intersect_nearest,tchain_idronaut_intersect_nearest,"temp",output_folder,cc=b)
except:
    log("Failed plotting verification Idronaut vs Temperature Chain")

10:57:52.354473   plotting verification Idronaut vs Temperature Chain
Figure saved: Idronaut vs Temperature Chain temperature depth.pdf
Figure saved: Idronaut vs Temperature Chain temperature date.pdf


### Thetis vs Temperature chain

In [30]:
log("plotting verification Thetis vs Temperature Chain")
try:
    for b in cc_dict:
        ver_plotting_tchain(thetis_tchain_intersect_nearest,tchain_thetis_intersect_nearest,"temp",output_folder,cc=b)
except:
    log("Failed plotting verification Thetis vs Temperature Chain")

11:01:28.158546   plotting verification Thetis vs Temperature Chain
Figure saved: Thetis CTD vs Temperature Chain temperature depth.pdf
Figure saved: Thetis CTD vs Temperature Chain temperature date.pdf


### Thetis vs Idronaut

In [31]:
log("plotting verification Thetis vs Idronaut")
try:
    for i in idronaut_thetis_dict:
        for b in cc_dict:
            ver_plotting(thetis_idronaut_intersect_nearest,idronaut_thetis_intersect_nearest,output_folder=output_folder,var1=i,var2=i,cc=b)
except:
    log("Failed plotting verification Thetis vs Idronaut")

11:05:24.071818   plotting verification Thetis vs Idronaut
Figure saved: Thetis CTD vs Idronaut Chlorophyll a depth.pdf
Figure saved: Thetis CTD vs Idronaut Chlorophyll a date.pdf
Figure saved: Thetis CTD vs Idronaut temperature depth.pdf
Figure saved: Thetis CTD vs Idronaut temperature date.pdf
Figure saved: Thetis CTD vs Idronaut salinity depth.pdf
Figure saved: Thetis CTD vs Idronaut salinity date.pdf
Figure saved: Thetis CTD vs Idronaut conductivity depth.pdf
Figure saved: Thetis CTD vs Idronaut conductivity date.pdf


### Thetis vs PP Mooring

In [34]:
log("plotting verification Thetis vs PP Mooring")
try:
    for b in cc_dict:
        ver_plotting_pp(thetis_pp_par_intersect_nearest,pp_par_thetis_intersect_nearest,"par",output_folder=output_folder,cc=b)
        ver_plotting_pp(thetis_pp_do_intersect_nearest,pp_do_thetis_intersect_nearest,"do",output_folder=output_folder,cc=b)
except:
    log("Failed plotting verification Thetis vs PP Mooring")

11:10:30.184006   plotting verification Idronaut vs PP Mooring
Figure saved: Thetis CTD vs PP Mooring photosynthetically active radiation 5, 10, 30m depth.pdf
Figure saved: Thetis CTD vs PP Mooring dissolvedoxygen 5, 10, 30m depth.pdf
Figure saved: Thetis CTD vs PP Mooring photosynthetically active radiation 5, 10, 30m date.pdf
Figure saved: Thetis CTD vs PP Mooring dissolvedoxygen 5, 10, 30m date.pdf


### Manual CTD vs PP Mooring DO

In [35]:
log("plotting verification Manual CTD vs PP Mooring")
try:
    for b in cc_dict:
        ver_plotting_pp(ctd_pp_do_intersect_nearest,pp_do_ctd_intersect_nearest,"do",output_folder=output_folder,cc=b)
except:
    log("Failed plotting verification Manual CTD vs PP Mooring")

11:46:22.076771   plotting verification Manual CTD vs PP Mooring
Figure saved: LéXPLORE CTD vs PP Mooring oxygen concentration 5, 10, 30m depth.pdf
Figure saved: LéXPLORE CTD vs PP Mooring oxygen concentration 5, 10, 30m date.pdf


In [36]:
# end of verification